In [33]:
import pandas as pd
df = pd.read_table("epa-http.txt")
df.columns = ['column']
s = df['column'].str.split(' ').apply(Series, 1).stack()

NameError: name 'Series' is not defined

In [31]:
print(s[0:10])

0    [query2.lycos.cs.cmu.edu, [29:23:53:36], "GET,...
1    [tanuki.twics.com, [29:23:53:53], "GET, /News....
2    [wpbfl2-45.gate.net, [29:23:54:15], "GET, /, H...
3    [wpbfl2-45.gate.net, [29:23:54:16], "GET, /ico...
4    [wpbfl2-45.gate.net, [29:23:54:18], "GET, /log...
5    [140.112.68.165, [29:23:54:19], "GET, /logos/u...
6    [wpbfl2-45.gate.net, [29:23:54:19], "GET, /log...
7    [wpbfl2-45.gate.net, [29:23:54:19], "GET, /ico...
8    [wpbfl2-45.gate.net, [29:23:54:19], "GET, /log...
9    [tanuki.twics.com, [29:23:54:19], "GET, /docs/...
Name: column, dtype: object


In [32]:
print(s[5:5])

Series([], Name: column, dtype: object)
